### How many games does a team needs to win to be crowned champions?

First we will need to import the standing tables

In [1]:
import re
import numpy as np
import pandas as pd
from library import (
    get_squad_as_index,
    remove_unnamed_cols,
    clean_opp_df,
    clean_main_df,
    merge_dfs,
    get_single_season_league_data,
    edit_gk_tables,
)


In [2]:
df = get_single_season_league_data(country="England", tier=1, year=2021)
df.keys()

dict_keys(['standings_table', 'home_away', 'standard_data', 'gk_overall', 'shooting', 'passing', 'pass_types', 'gca', 'defensive_actions', 'possession', 'other'])

In [3]:
df.get("gk_overall")

Performance                                  Penalty_Kicks  \
                         GA  GA90 SoTA Saves Save%  CS   CS%         PKatt   
Arsenal                  48  1.26  147   100  70.7  13  34.2             6   
Aston Villa              54  1.42  149    96  66.4  11  28.9             6   
Brentford                56  1.47  179   125  71.5   9  23.7             5   
Brighton                 44  1.16  146   102  71.9  11  28.9             4   
Burnley                  53  1.39  178   127  73.0   9  23.7             5   
Chelsea                  33  0.87  115    83  76.5  16  42.1             6   
Crystal Palace           46  1.21  144    99  70.1  12  31.6             4   
Everton                  66  1.74  188   127  66.0   8  21.1             2   
Leeds United             79  2.08  224   146  67.0   5  13.2             5   
Leicester City           59  1.55  192   132  72.9   7  18.4             9   
Liverpool                26  0.68  107    82  75.7  21  55.3             0   
Manchester City          26  0.68   85    59  70.6  21  55.3             1   
Manchester Utd           57  1.50  180   124  70.0   8  21.1             5   
Newcastle Utd            62  1.63  165   104  65.5   8  21.1             6   
Norwich City             84  2.21  222   139  66.2   6  15.8            12   
Southampton              67  1.76  183   118  64.5   8  21.1             2   
Tottenham                40  1.05  137    98  72.3  16  42.1             2   
Watford                  77  2.03  188   113  62.2   4  10.5             7   
West Ham                 51  1.34  165   113  70.3   8  21.1             6   
Wolves                   43  1.13  167   125  79.6  11  28.9            10   

                          ... Performance                                     \
                PKA PKsv  ...    SoTA_Opp Saves_Opp Save%_Opp CS_Opp CS%_Opp   
Arsenal           5    0  ...       194.0     131.0      71.1   11.0    28.9   
Aston Villa       4    1  ...       162.0     112.0      69.8    8.0    21.1   
Brentford         5    0  ...       147.0     101.0      71.4   12.0    31.6   
Brighton          3    0  ...       146.0     105.0      74.0   13.0    34.2   
Burnley           5    0  ...       120.0      88.0      72.5   16.0    42.1   
Chelsea           6    0  ...       209.0     133.0      67.5    4.0    10.5   
Crystal Palace    3    1  ...       147.0      96.0      70.1   11.0    28.9   
Everton           2    0  ...       136.0      94.0      72.8   12.0    31.6   
Leeds United      5    0  ...       147.0     105.0      74.8   13.0    34.2   
Leicester City    7    2  ...       166.0     104.0      63.9    6.0    15.8   
Liverpool         0    0  ...       245.0     150.0      64.5    1.0     2.6   
Manchester City   1    0  ...       246.0     148.0      62.6    4.0    10.5   
Manchester Utd    3    2  ...       182.0     126.0      70.3    9.0    23.7   
Newcastle Utd     5    1  ...       145.0     104.0      71.7    9.0    23.7   
Norwich City      9    1  ...       107.0      86.0      81.3   22.0    57.9   
Southampton       2    0  ...       165.0     124.0      76.4    9.0    23.7   
Tottenham         2    0  ...       190.0     127.0      66.3   10.0    26.3   
Watford           6    1  ...       125.0      91.0      73.6   18.0    47.4   
West Ham          2    3  ...       148.0      88.0      62.2    8.0    21.1   
Wolves            9    0  ...         NaN       NaN       NaN    NaN     NaN   

                Penalty_Kicks                                     
                    PKatt_Opp PKA_Opp PKsv_Opp PKm_Opp Save%_Opp  
Arsenal                   8.0     5.0      3.0     0.0      37.5  
Aston Villa               3.0     3.0      0.0     0.0       0.0  
Brentford                 6.0     6.0      0.0     0.0       0.0  
Brighton                  7.0     4.0      1.0     2.0      20.0  
Burnley                   2.0     1.0      0.0     1.0       0.0  
Chelsea                   9.0     8.0      1.0     0.0      11.1  
Cr